# Mapas

In [ ]:
import pandas as pd
import geopandas
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt

Leitura de datasets (od e distritos)

In [ ]:
od23_path = "../data/OD/ODS/od23_all.csv" 
od = pd.read_csv(od23_path)

In [ ]:
od_districts_shapefile_path = "../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Distritos_2023_region.shp" 
districts = geopandas.read_file(od_districts_shapefile_path)

In [ ]:
print(districts.columns, districts.crs)

In [ ]:
# districts.explore("Area_ha", legend=True)

Filtro de pedestres

In [ ]:
pedestrians = od.query('MODOPRIN == 17')
pedestrians.head()

GeoDataFrame com pontos(coordenadas) de origens de pedestres

In [ ]:
pedestrians_origin_gdf = geopandas.GeoDataFrame(
    pedestrians,
    geometry=geopandas.points_from_xy(pedestrians["CO_O_Y"], pedestrians["CO_O_X"]),
    crs="EPSG:4326"
)

Conversão de CRS dos dataframes para um CRS comum

In [ ]:
districts = districts.to_crs(epsg=31983)
pedestrians_origin_gdf = pedestrians_origin_gdf.to_crs(epsg=31983)
# districts

Join entre distristos e origens de pedestres

In [ ]:
origin_join = geopandas.sjoin(pedestrians_origin_gdf, districts, "inner")
origin_join

Mapa com pontos de origens de pedestres

In [ ]:
origin_join.plot()

Mapa interativo com pontos de origem com overlay dos distritos

In [ ]:
m = districts.explore(edgecolor="black", color=None, tooltip="NomeDistri") 
# origin_join.explore(m=m)

Mapa com pontos de origem e distritos

In [ ]:
ax = districts.plot(edgecolor='black', figsize=(10,10))
pedestrians_origin_gdf.plot(ax=ax, color='red', markersize=5)
# plt.show()

GeoDataFrame com LineStrings entre origens e destinos de pedestres

Colunas/campos importantes:
- pedestrians["CO_O_Y"], pedestrians["CO_O_X"] - coordenadas (lat, lon) de origem
- pedestrians["CO_D_Y"], pedestrians["CO_D_X"] - coordenadas (lat, lon) de destino

In [ ]:
pedestrians_lines_gdf = geopandas.GeoDataFrame(
    pedestrians,
    geometry=[
        LineString([
            (row["CO_O_X"], row["CO_O_Y"]),
            (row["CO_D_X"], row["CO_D_Y"])
        ])
        for _, row in pedestrians.iterrows()
    ],
    crs="EPSG:4326"
)
pedestrians_lines_gdf.head()

In [ ]:
districts_4326 = districts.to_crs("EPSG:4326")
fig, ax = plt.subplots(figsize=(10, 10))
districts_4326.plot(ax=ax, edgecolor="black", color="none")
pedestrians_lines_gdf.plot(ax=ax, color="red", linewidth=0.2)
plt.show()

In [ ]:
m = districts.to_crs("EPSG:4326").explore(edgecolor="black", color=None, tooltip="NomeDistri")
pedestrians_lines_gdf.explore(m=m)